### Import our dependencies

In [1]:
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mc
import statsmodels.formula.api as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import json

### Read our CSV

In [2]:
us_df = pd.read_csv('USvideos.csv')

In [5]:
us_df.head(10)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
5,gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...
7,nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....
8,jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th..."
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo..."


In [6]:
# Remove unecessary columns
del_col = ['video_id','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
us_df = us_df.drop(del_col, axis=1)

### Read json file 

In [8]:
cat_json = pd.read_json("US_category_id.JSON")
with open("US_category_id.JSON") as f:
    data = json.load(f)
    print(data)

{'kind': 'youtube#videoCategoryListResponse', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"', 'items': [{'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', 'id': '1', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Film & Animation', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', 'id': '2', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Autos & Vehicles', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"', 'id': '10', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Music', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"', 'id': '15', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Pets & Animals', 'assignable': True}}, {'kind': 'yout

In [9]:
cat_dict = [{'id': item['id'], 'title': item['snippet']['title']} for item in cat_json['items']]

In [10]:
us_cat_df = pd.DataFrame(cat_dict)
categories = us_cat_df.rename(index=str, columns={"id": "category_id", "title": "category"})
categories.head(3)

,category_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music


In [11]:
us_df['tags'].head(50)

0                                       SHANtell martin
1     last week tonight trump presidency|"last week ...
2     racist superman|"rudy"|"mancuso"|"king"|"bach"...
3     rhett and link|"gmm"|"good mythical morning"|"...
4     ryan|"higa"|"higatv"|"nigahiga"|"i dare you"|"...
5     ijustine|"week with iPhone X"|"iphone x"|"appl...
6     SNL|"Saturday Night Live"|"SNL Season 43"|"Epi...
7     5 Ice Cream Gadgets|"Ice Cream"|"Cream Sandwic...
8     Trailer|"Hugh Jackman"|"Michelle Williams"|"Za...
9     vox.com|"vox"|"explain"|"shift change"|"future...
10    NFL|"Football"|"offense"|"defense"|"afc"|"nfc"...
11    The Walking Dead|"shiva"|"tiger"|"king ezekiel...
12    marshmello|"blocks"|"marshmello blocks"|"block...
13    nowthis|"nowthis world"|"world news"|"nowthis ...
14    shopping for new fish|"new fish"|"aquarium fis...
15    Robots|"Boston Dynamics"|"SpotMini"|"Legged Lo...
16    pacific rim|"pacific rim 2"|"pacific rim seque...
17    TED|"TED-Ed"|"TED Education"|"TED Ed"|"Hil

In [12]:
us_df['tags'].unique().shape[0]

6055

In [13]:
all_tags = []
for tgs in us_df['tags']:
    if tgs == '[none]': continue
    all_tags.extend(tgs.split('|'))
tags_freq = pd.Series(all_tags).value_counts().to_dict()

In [14]:
sorted(tags_freq.items(), key=lambda x: x[1], reverse=True)[:100]

[('"funny"', 3584),
 ('"comedy"', 2863),
 ('"how to"', 1560),
 ('"Pop"', 1273),
 ('"2018"', 1251),
 ('"music"', 1221),
 ('"humor"', 1171),
 ('"food"', 1134),
 ('"science"', 1087),
 ('"review"', 1002),
 ('"news"', 941),
 ('"vlog"', 928),
 ('"makeup"', 924),
 ('"celebrity"', 881),
 ('"tutorial"', 864),
 ('"live"', 862),
 ('"comedian"', 861),
 ('"interview"', 845),
 ('"video"', 799),
 ('"cooking"', 780),
 ('"television"', 779),
 ('"celebrities"', 767),
 ('"fun"', 737),
 ('"movie"', 732),
 ('"cute"', 726),
 ('"trailer"', 711),
 ('"Comedy"', 710),
 ('"family friendly"', 694),
 ('"official"', 690),
 ('"hollywood"', 675),
 ('"funny video"', 673),
 ('"NBC"', 666),
 ('"Records"', 647),
 ('"entertainment"', 647),
 ('"challenge"', 641),
 ('"2017"', 641),
 ('"sports"', 639),
 ('"education"', 635),
 ('"jokes"', 626),
 ('"animation"', 608),
 ('"late night"', 607),
 ('"recipe"', 605),
 ('"beauty"', 591),
 ('"animals"', 589),
 ('"basketball"', 588),
 ('"makeup tutorial"', 583),
 ('"sketch"', 575),
 ('

In [15]:
vector = CountVectorizer()
counts = vector.fit_transform(us_df['tags'].values)

In [16]:
# Define the model
NB_Model = MultinomialNB()
targets = us_df['category_id'].values
NB_Model.fit(counts,targets)

MultinomialNB()

In [17]:
X= counts
y= targets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1)

NBtest = MultinomialNB().fit(X_train, y_train)
nb_predictions = NBtest.predict(X_test)
acc_nb = NBtest.score(X_test, y_test)
print('The Naive Bayes Algorithm scored an accuracy of', acc_nb)

The Naive Bayes Algorithm scored an accuracy of 0.9045177045177045


In [18]:
X.shape

(40949, 23483)

In [19]:
y.shape

(40949,)

In [20]:
# Choose random sample of tag words
Tags = ["Billie Eilish",
        "SNL",
        "you should be loved",
        "NBA",
        "react",
        "marvel",
        "Taylor Swift",
        "BLACKPINK",
        "BTS",
        "penguins",
        "vaccines",
        "donald trump",
        "Tryguys",
        "TED TALK",
        "Dark Souls"
         ]

In [21]:
tags_counts = vector.transform(Tags)
prediction = NB_Model.predict(tags_counts)
prediction

array([10, 24, 10, 17, 24, 24, 10, 24, 10, 15, 28, 25, 24, 27, 20],
      dtype=int64)

In [22]:
category_list = []
for category_id in prediction:
    matching = [x for x in cat_dict if x["id"] == str(category_id)]
    if matching:
        category_list.append(matching[0]["title"])

In [23]:
tags_df = []
for i in range(0, len(Tags)):
    tags_cat = {'tags': Tags[i],  'category_id': category_list[i]}
    tags_df.append(tags_cat)

In [24]:
prediction_df = pd.DataFrame(prediction)
tags_df = pd.DataFrame(tags_df)
df = pd.concat([prediction_df, tags_df], axis=1)
df.columns = (['Categ_ID','Video Tag', 'Category'])
us_pred_df = df.drop(['Categ_ID'],axis=1)
cols = us_pred_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
us_pred_df= us_pred_df[cols]

In [25]:
us_pred_df

,Category,Video Tag
0,Music,Billie Eilish
1,Entertainment,SNL
2,Music,you should be loved
3,Sports,NBA
4,Entertainment,react
5,Entertainment,marvel
6,Music,Taylor Swift
7,Entertainment,BLACKPINK
8,Music,BTS
9,Pets & Animals,penguins
